In [1]:
import sys
parentdir = "../.."
sys.path.insert(0, parentdir)

import requests
from os import path
from tqdm.notebook import tqdm
import pandas as pd
import time

from metrics.WebResource import WebResource
from metrics.FAIRMetricsFactory import FAIRMetricsFactory
from metrics.AbstractFAIRMetrics import AbstractFAIRMetrics

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST driver version for 101.0.4951
[WDM] - Driver [/home/trosnet/.wdm/drivers/chromedriver/linux64/101.0.4951.41/chromedriver] found in cache


In [2]:
from rdflib import ConjunctiveGraph, Namespace, URIRef
from rdflib.namespace import RDF, RDFS

schema = Namespace("http://schema.org/")

In [3]:
TIMEOUT = (10, 300)
#NB = 500
NB = 600
OUTPUT_DIR = "../results/dryad_data/"

def dryadRequest(url):
    print("REST request to dryad...")
    # rest request
    while True:
        try:
            response = requests.get(url, timeout=TIMEOUT)
            break
        except requests.exceptions.SSLError:
            print("Error, retrying...")
            time.sleep(5)
        except requests.exceptions.Timeout:
            print("Timeout error, retrying...")
            time.sleep(5)

    return response

def dryadParser(response):
    print("Parsing result...")
    json_response = response.json()
    next_page = "https://datadryad.org" + json_response["_links"]["next"]["href"]
    count_step = int(json_response["count"] + 5)
    print(next_page)

    
    dois_list = []
    count = 0
    while True:
        for element in json_response["_embedded"]["stash:datasets"]:
            identifier = element["identifier"].replace("doi:", "https://doi.org/")
            dois_list.append(identifier)

        count += count_step
        if count > NB:
            break
        response = dryadRequest(next_page)
        json_response = response.json()
        next_page = "https://datadryad.org" + json_response["_links"]["next"]["href"]
        print(next_page)
        

    print("Retrieved " + str(len(dois_list)) + " DOIs")
    return dois_list
    

In [4]:
response = dryadRequest("https://datadryad.org/api/v2/datasets?per_page=100")
dois_list = dryadParser(response)


print(len(dois_list))

REST request to dryad...
Parsing result...
https://datadryad.org/api/v2/datasets?page=2&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=3&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=4&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=5&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=6&per_page=100
REST request to dryad...
https://datadryad.org/api/v2/datasets?page=7&per_page=100
Retrieved 600 DOIs
600


In [5]:
if "https://doi.org/10.5061/dryad.7b2s3" in dois_list:
    dois_list.remove("https://doi.org/10.5061/dryad.7b2s3")
    print("Removed")
if "https://doi.org/10.5061/dryad.jd5716f" in dois_list:
    dois_list.remove("https://doi.org/10.5061/dryad.jd5716f")
    print("Removed")
print(type(dois_list))

Removed
<class 'list'>


In [6]:
import random

random.seed(10)

samples_dois_list = random.sample(dois_list, 500)

splitedSize = 100
samples_chunks = [samples_dois_list[x:x+splitedSize] for x in range(0, len(samples_dois_list), splitedSize)]

In [7]:
print(len(samples_dois_list))

500


In [8]:
# open file in write mode
with open("../results/dryad_data/url_list", 'w+') as fp:
    for item in samples_dois_list:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

#open and read the file
with open("../results/dryad_data/url_list") as f:
    lines = [line.strip() for line in f]
    samples_dois_list = lines

Done


In [9]:
metrics_collection = []
metrics_collection.append(FAIRMetricsFactory.get_A11(None))
metrics_collection.append(FAIRMetricsFactory.get_F1A(None))
metrics_collection.append(FAIRMetricsFactory.get_F1B(None))
metrics_collection.append(FAIRMetricsFactory.get_F2A(None))
metrics_collection.append(FAIRMetricsFactory.get_F2B(None))
metrics_collection.append(FAIRMetricsFactory.get_I1(None))
metrics_collection.append(FAIRMetricsFactory.get_I2(None))
metrics_collection.append(FAIRMetricsFactory.get_I3(None))
metrics_collection.append(FAIRMetricsFactory.get_R11(None))
metrics_collection.append(FAIRMetricsFactory.get_R12(None))
metrics_collection.append(FAIRMetricsFactory.get_R13(None))

df_columns = ['ID']
for m in metrics_collection:
    df_columns.append(m.get_principle_tag())

In [10]:
exec_time_df = pd.DataFrame(columns=df_columns)

def eval_metrics(web_res):
    metrics_collection = []
    metrics_collection.append(FAIRMetricsFactory.get_A11(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F1A(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F1B(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F2A(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_F2B(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_I1(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_I2(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_I3(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_R11(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_R12(web_res))
    metrics_collection.append(FAIRMetricsFactory.get_R13(web_res))
    
    row = {"ID":web_res.get_url()}
    row_time = {"ID":web_res.get_url()}
    for m in metrics_collection:
        ts1 = time.time()
        e = m.evaluate()
        duration = round((time.time() - ts1), 2)
        if e is not None:
            row[m.get_principle_tag()] = e.get_score()
            row_time[m.get_principle_tag()] = duration
    
    return row, row_time


In [11]:
def mass_eval(samples):
    evals = []
    exec_time = []

    for sample in tqdm(samples):
        print(sample)
        wr = WebResource(url=sample)
        row, row_time = eval_metrics(wr)
        evals.append(row)
        exec_time.append(row_time)
        
    
    return evals, exec_time

In [ ]:
import logging
logging.getLogger().setLevel(logging.ERROR)

i = 0
for c in tqdm(samples_chunks):
    i += 1
    df = pd.DataFrame()
    df_time = pd.DataFrame()
    
    evals, exec_time = mass_eval(c)
    df = pd.concat([df, pd.DataFrame.from_records(evals)])
    df_time = pd.concat([df_time, pd.DataFrame.from_records(exec_time)])
    
    df.to_csv(OUTPUT_DIR + "FC_results_DRYAD_data_"+str(i)+".csv")
    df_time.to_csv(OUTPUT_DIR + "exec_time_DRYAD_data_"+str(i)+".csv")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

https://doi.org/10.5061/dryad.2nh26


/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datadryad.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<class 'dict'>
{'@context': 'http://schema.org', '@type': 'Dataset', '@id': 'https://doi.org/10.5061/dryad.2nh26', 'name': 'Xianella: a new mat-forming calcified cyanobacterium from the Middle–Late Ordovician of North China', 'description': ['Xianella hongii gen. et sp. nov. is described from the Middle–Late Ordovician of Shaanxi, China and interpreted as a calcified cyanobacterial sheath. Xianella filaments formed cable-like strands that constructed thick fenestral layers. The specimens occur in metre-sized limestone blocks, possibly derived from local collapse of a reefal platform margin. In combination with micrite, some of which is intraclastic and peloidal, Xianella created thick and extensive stacks of layered calcified fenestral fabric that appear to be synsedimentarily calcified open-frame mat deposits. The fenestrae range from small, laminose and very irregular, to large equidimensional areas ~2 cm across. Fenestrae with rounded outlines resemble primary gas bubbles observed i

...

/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datadryad.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
INFO:c275ae6b-7f87-4805-a9a8-697f3b4a6d79:Checking if the URL uses HTTP protocol
INFO:c275ae6b-7f87-4805-a9a8-697f3b4a6d79:The resource use HTTP protocol
INFO:13da3279-553c-4787-a463-f37db6781006:Checking if the URL is reachable, status code: 200
INFO:13da3279-553c-4787-a463-f37db6781006:Status code is OK, meaning th

<class 'dict'>
{'@context': 'http://schema.org', '@type': 'Dataset', '@id': 'https://doi.org/10.5061/dryad.tv5v1v26', 'name': 'Diversity patterns of uncultured Haptophytes unravelled by pyrosequencing in Naples Bay', 'description': ['Haptophytes are a key phylum of marine protists, including ~300 described morphospecies and 80 morphogenera. We used 454 pyrosequencing on large subunit ribosomal DNA (LSU rDNA) fragments to assess the diversity from size-fractioned plankton samples collected in the Bay of Naples. One group-specific primer set targeting the LSU rDNA D1/D2 region was designed to amplify Haptophyte sequences from nucleic acid extracts (total DNA or RNA) of two size fractions (0.8–3 or 3–20 μm) and two sampling depths [subsurface, at 1 m, or deep chlorophyll maximum (DCM) at 23 m]. 454 reads were identified using a database covering the entire Haptophyta diversity currently sequenced. Our data set revealed several hundreds of Haptophyte clusters. However, most of these cluste

/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datadryad.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
INFO:cc36332b-2b9a-4f68-87b1-279afb2ad6f0:Checking if the URL uses HTTP protocol
INFO:cc36332b-2b9a-4f68-87b1-279afb2ad6f0:The resource use HTTP protocol
INFO:d4bfa944-a679-435c-b3c0-75bb131103fc:Checking if the URL is reachable, status code: 200
INFO:d4bfa944-a679-435c-b3c0-75bb131103fc:Status code is OK, meaning th

<class 'dict'>
{'@context': 'http://schema.org', '@type': 'Dataset', '@id': 'https://doi.org/10.5061/dryad.r2b1h', 'name': 'Testing adaptive hypotheses of convergence with functional landscapes: a case study of bone-cracking hypercarnivores', 'description': ['Morphological convergence is a well documented phenomenon in mammals, and adaptive explanations are commonly employed to infer similar functions for convergent characteristics. I present a study that adopts aspects of theoretical morphology and engineering optimization to test hypotheses about adaptive convergent evolution. Bone-cracking ecomorphologies in Carnivora were used as a case study. Previous research has shown that skull deepening and widening are major evolutionary patterns in convergent bone-cracking canids and hyaenids. A simple two-dimensional design space, with skull width-to-length and depth-to-length ratios as variables, was used to examine optimized shapes for two functional properties: mechanical advantage (MA) 

/home/trosnet/miniconda3/envs/fair-checker-env/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doi.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [ ]:
len(df)

In [ ]:
df_time

In [ ]:
import glob

all_files = glob.glob(OUTPUT_DIR + "FC_results_DRYAD_data_*.csv")

li = []

for filename in tqdm(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
from upsetplot import generate_counts, generate_samples, UpSet,plot
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200 

In [ ]:
from upsetplot import from_contents

A11 = df[((df["A1.1"].astype(int) > 0))]["ID"]

F1A = df[((df["F1A"].astype(int) > 0))]["ID"]
F1B = df[((df["F1B"].astype(int) > 0))]["ID"]
F2A = df[((df["F2A"].astype(int) > 0))]["ID"]
F2B = df[((df["F2B"].astype(int) > 0))]["ID"]

I1 = df[((df["I1"].astype(int) > 0))]["ID"]
I2 = df[((df["I2"].astype(int) > 0))]["ID"]
I3 = df[((df["I3"].astype(int) > 0))]["ID"]

R11 = df[((df["R1.1"].astype(int) > 0))]["ID"]
R12 = df[((df["R1.2"].astype(int) > 0))]["ID"]
R13 = df[((df["R1.3"].astype(int) > 0))]["ID"]

df_upset = from_contents({
    'A11': A11,
    'F1A': F1A, 
    'F1B': F1B, 
    'F2A': F2A, 
    'F2B': F2B, 
    'I1': I1, 
    'I2': I2, 
    'I3': I3, 
    'R11': R11, 
    'R12': R12,
    'R13': R13,
})
df_upset

In [ ]:
upset = UpSet(df_upset, subset_size='count', show_counts=True, sort_categories_by=None, orientation='horizontal')

params = {'legend.fontsize': 8}
with plt.rc_context(params):
    upset.plot()
plt.suptitle("Datasets from Dryad matching metrics")
plt.savefig(OUTPUT_DIR + 'DRYAD_all_data.png', format="png")
plt.show()